In [ ]:
import ollama

In [ ]:
def pull_model_if_needed(model_name):
    """
    Attempts to pull the model. Ollama handles checking if it exists.
    Returns True on success, raises an exception on failure.
    """
    try:
        ollama.pull(model=model_name)
        print(f"Model '{model_name}' pulled successfully/already existed.")
        return True
    except Exception as e:
        error_message = f"Error pulling model '{model_name}': {e}"
        print(error_message)
        raise  # Re-raise the exception for the caller to handle

In [ ]:
def _run_prompt(model, prompt, prompt_type):
    """Internal helper function to run a prompt."""
    print(f"--- {prompt_type} Prompting ---")
    print(f"Prompt:\n{prompt}\n")
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt,
                },
            ]
        )
        result = response['message']['content'].strip()
        print(f"Response:\n{result}\n")
        return result
    except Exception as e:
        print(f"Error during {prompt_type.lower()} prompting: {e}\n")
        return None

In [ ]:
def run_zero_shot(model, prompt):
    """Runs zero-shot prompting on the specified model."""
    return _run_prompt(model, prompt, "Zero-shot")

In [ ]:
def run_chain_of_thought(model, prompt):
    """Runs chain-of-thought prompting on the specified model."""
    return _run_prompt(model, prompt, "Chain-of-Thought")

In [ ]:
def run_few_shot(model, prompt, examples):
    """Runs few-shot prompting on the specified model and displays examples."""
    print("--- Few-shot Prompting ---")
    print("Examples:")
    for example in examples:
        print(f"  Prompt: {example['prompt']}")
        print(f"  Completion: {example['completion']}")
    print("-" * 20)
    print(f"Prompt:\n{prompt}\n")

    messages = []
    for example in examples:
        messages.append({'role': 'user', 'content': example['prompt']})
        messages.append({'role': 'assistant', 'content': example['completion']})
    messages.append({'role': 'user', 'content': prompt})

    try:
        response = ollama.chat(
            model=model,
            messages=messages
        )
        result = response['message']['content'].strip()
        print(f"Response:\n{result}\n")
        return result
    except Exception as e:
        print(f"Error during few-shot prompting: {e}\n")
        return None

In [ ]:
def run_ollama_experiment(model, task_description, zero_shot_prompt, few_shot_examples=None, chain_of_thought_prompt=None):
    """
    Facilitates an experiment with different prompting techniques on an Ollama model.

    Args:
        model (str): The name of the Ollama model to use.
        task_description (str): A brief description of the task.
        zero_shot_prompt (str): The prompt for the zero-shot experiment.
        few_shot_examples (list, optional): A list of dictionaries, where each dictionary
                                            has 'prompt' and 'completion' keys representing
                                            an example. Defaults to None.
        chain_of_thought_prompt (str, optional): The prompt for the chain-of-thought
                                                experiment. Defaults to None.

    Returns:
        dict: A dictionary containing the results of each prompting technique.
    """

    print(f"\n--- Experimenting with model: {model} ---\nTask: {task_description}\n")

    results = {
        "task_description": task_description,
        "model": model,
        "zero_shot": run_zero_shot(model, zero_shot_prompt),
        "few_shot": run_few_shot(model, zero_shot_prompt, few_shot_examples) if few_shot_examples else None,
        "chain_of_thought": run_chain_of_thought(model, chain_of_thought_prompt) if chain_of_thought_prompt else None,
    }

    return results


In [ ]:
model_name = "gemma2"  # Replace with your desired default model

print("Starting experiment...pulling models as required...")
try:
    pull_model_if_needed(model_name)
    model_to_use = model_name
    print(f"\nUsing model: {model_to_use}\n")

    # Task: Summarize a paragraph of text
    summary_task_description = "Summarize a paragraph of text."
    paragraph_to_summarize = """
    The quick brown rabbit jumps over the lazy frogs with no effort. The frogs,
    startled by the sudden movement, croak loudly in protest. The rabbit,
    unfazed by the noise, continues its journey through the lush green meadow,
    eager to reach its burrow before sunset. The gentle breeze rustles the leaves
    in the nearby trees, creating a soothing natural symphony.
    """
    summary_zero_shot_prompt = f"Summarize the following paragraph:\n\n{paragraph_to_summarize}"
    summary_few_shot_examples = [
        {
            "prompt": "Summarize: The cat sat on the mat.",
            "completion": "A cat is sitting on a mat."
        },
        {
            "prompt": "Summarize: The dog barked loudly at the mailman.",
            "completion": "A dog barked at the person delivering mail."
        }
    ]
    summary_chain_of_thought_prompt = f"Let's think step-by-step. First, I will identify the main entities and actions in the paragraph. Then, I will condense this information into a shorter summary. Summarize the following paragraph:\n\n{paragraph_to_summarize}"

    summary_results = run_ollama_experiment(
        model=model_to_use,
        task_description=summary_task_description,
        zero_shot_prompt=summary_zero_shot_prompt,
        few_shot_examples=summary_few_shot_examples,
        chain_of_thought_prompt=summary_chain_of_thought_prompt
    )

    # Task: Solve a simple logic puzzle
    logic_task_description = "Solve a simple logic puzzle."
    logic_puzzle = """
    Alice, Bob, and Carol are sitting in a row. Bob is not at either end.
    Alice is to the left of Bob. Who is sitting on the right end?
    """
    logic_zero_shot_prompt = f"Solve this logic puzzle:\n\n{logic_puzzle}"
    logic_few_shot_examples = [
        {
            "prompt": "Solve: If all birds can fly and a penguin is a bird, can a penguin fly?",
            "completion": "Let's think step-by-step. All birds can fly. A penguin is a bird. Therefore, a penguin can fly. Final Answer: Yes."
        }
    ]
    logic_chain_of_thought_prompt = f"Let's think step-by-step. I will analyze the clues provided. Based on these clues, I can deduce the order. Solve this logic puzzle:\n\n{logic_puzzle}"

    logic_results = run_ollama_experiment(
        model=model_to_use,
        task_description=logic_task_description,
        zero_shot_prompt=logic_zero_shot_prompt,
        few_shot_examples=logic_few_shot_examples,
        chain_of_thought_prompt=logic_chain_of_thought_prompt
    )

    print("\n--- Experiment Finished ---")

    print("\n--- Results from Experiments ---")
    print(summary_results)
    print(logic_results)

except Exception as e:
    print(f"Exiting due to an error: {e}")